In [120]:
![imagen.png](attachment:imagen.png)

"[imagen.png]" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [36]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Dataset

In [37]:
from sklearn import datasets

cancer = datasets.load_breast_cancer()
df_cancer = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
df_cancer['target'] = cancer.target
df_cancer

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


In [38]:
df_cancer.groupby("target").size()

target
0    212
1    357
dtype: int64

In [60]:
df_cancer.isnull().values.any()

False

In [39]:
X = df_cancer.drop(df_cancer.columns[-1], axis=1)
y = df_cancer['target']

# Modelo

In [84]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import RobustScaler

model=LogisticRegression(solver='liblinear')
# model=SVC(kernel='linear') 
pipeline = make_pipeline(\
                         RobustScaler(),
                         LogisticRegression(solver='liblinear'))

# Aplicación convencional

In [85]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=100)
# model.fit(X_train, y_train)
# pred = model.predict(X_test)
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)

pd.DataFrame({'Accuracy on Training':[accuracy_score(y_train, pipeline.predict(X_train))], 
             'Accuracy on Test':[accuracy_score(y_test, pred)]})

,Accuracy on Training,Accuracy on Test
0,0.991209,0.982456


# Cross validation
### KfoldShuffle

In [89]:
from sklearn.model_selection import KFold

folds = KFold(n_splits = 5, shuffle=True, random_state=100)
accuracy_df = pd.DataFrame()
for train_index, test_index in folds.split(df_cancer):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]
    
    # model.fit(X_train, y_train)
    # df = {'Accuracy on Training':accuracy_score(y_train, model.predict(X_train)), 
    #       'Accuracy on Test':accuracy_score(y_test, model.predict(X_test))}
    pipeline.fit(X_train, y_train)
    df = {'Accuracy on Training':accuracy_score(y_train, pipeline.predict(X_train)), 
          'Accuracy on Test':accuracy_score(y_test, pipeline.predict(X_test))}
    accuracy_df=accuracy_df.append(df, ignore_index = True)
    
accuracy_df

,Accuracy on Training,Accuracy on Test
0,0.991209,0.982456
1,0.986813,0.973684
2,0.986813,0.991228
3,0.989011,0.982456
4,0.986842,0.982301


### StratifiedKFold

In [87]:
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits = 5, shuffle=True, random_state=100)
accuracy_df = pd.DataFrame()
for train_index, test_index in folds.split(X,y):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]
    
    # model.fit(X_train, y_train)
    # df = {'Accuracy on Training':accuracy_score(y_train, model.predict(X_train)), 
    #       'Accuracy on Test':accuracy_score(y_test, model.predict(X_test))}
    pipeline.fit(X_train, y_train)
    df = {'Accuracy on Training':accuracy_score(y_train, pipeline.predict(X_train)), 
          'Accuracy on Test':accuracy_score(y_test, pipeline.predict(X_test))}
    accuracy_df=accuracy_df.append(df, ignore_index = True)
    
accuracy_df

,Accuracy on Training,Accuracy on Test
0,0.984615,0.991228
1,0.989011,0.973684
2,0.984615,1.000000
3,0.989011,0.982456
4,0.991228,0.964602


# Grid Search

In [91]:
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression

# TODO: Cambiar a SVC
pipeline = make_pipeline(\
                         RobustScaler(),
                         SelectKBest(f_classif),
                         LogisticRegression(solver='liblinear'))

In [116]:
from sklearn.model_selection import GridSearchCV

# TODO: Checar qué son los atributos de param_grid
param_grid = {
    'selectkbest__k': [i+1 for i in range(30)],
    'logisticregression__class_weight': [None,'balanced'],
    'logisticregression__C': [.0001, .001, .01, .1, 1.0, 10.0, 100.00, 1000.0, 10000.0]
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=5,
                         scoring='accuracy', verbose=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=42
                                                   )
gridsearch.fit(X_train, y_train)

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('robustscaler', RobustScaler()),
                                       ('selectkbest', SelectKBest()),
                                       ('logisticregression',
                                        LogisticRegression(solver='liblinear'))]),
             param_grid={'logisticregression__C': [0.0001, 0.001, 0.01, 0.1,
                                                   1.0, 10.0, 100.0, 1000.0,
                                                   10000.0],
                         'logisticregression__class_weight': [None, 'balanced'],
                         'selectkbest__k': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                            12, 13, 14, 15, 16, 17, 18, 19, 20,
                                            21, 22, 23, 24, 25, 26, 27, 28, 29,
                                            30]},
             scoring='accuracy', verbose=1)

In [117]:
selector = gridsearch.best_estimator_.named_steps['selectkbest']
all_names = X_train.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]
print('Features selected:')
for name in selected_names:
    print(name)
print()
print('Features not selected:')
for name in unselected_names:
    print(name)

Features selected:
mean radius
mean texture
mean perimeter
mean area
mean smoothness
mean compactness
mean concavity
mean concave points
mean symmetry
mean fractal dimension
radius error
texture error
perimeter error
area error
smoothness error
compactness error
concavity error
concave points error
symmetry error
fractal dimension error
worst radius
worst texture
worst perimeter
worst area
worst smoothness
worst compactness
worst concavity
worst concave points
worst symmetry
worst fractal dimension

Features not selected:


In [118]:
y_pred = gridsearch.predict(X_test)
test_score = accuracy_score(y_test, y_pred)
print('Cross Validation Score:', gridsearch.best_score_)
print('Accuracy Score on test data set:', test_score)

Cross Validation Score: 0.9812038303693571
Accuracy Score on test data set: 0.986013986013986


In [119]:
best_parameters = gridsearch.best_params_
best_parameters

{'logisticregression__C': 1.0,
 'logisticregression__class_weight': None,
 'selectkbest__k': 30}